In [9]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pytrends.request import TrendReq
from pytrends import dailydata
import requests
import collections

from sklearn import linear_model
import statsmodels.api as sm

from backtesting import Backtest


In [13]:
with open("words.txt") as f:
    words = f.readlines()
words = [x.strip() for x in words]
# words = words[:5] + ["debt"]
# words = ["debt"]
#words = words[:10]

word_trends = {}

In [15]:
def getStats(key_word, start_year, start_month, end_year, end_month, index):
    trend = pd.DataFrame(dailydata.get_daily_data(key_word, start_year, start_month, end_year, end_month, wait_time = 0)[key_word])
    trend["change"] = trend.pct_change()

    # grab DJIA data
    index_data = yf.download(index, start = str(start_year)+"-"+str(start_month)+"-01", end = str(end_year)+"-"+str(end_month)+"-01")

    # join trend data with DJIA data
    joined = trend.merge(index_data, left_on = trend.index, right_on = index_data.index)
    joined = joined.rename(columns = {"key_0": "Date"})

    # grab the adj close price difference for each day
    #joined['Diff'] = joined['Adj Close'].diff()
    joined = joined.set_index(joined['Date'])

    joined["change moving avg"] = joined["change"].rolling("7d", min_periods = 1).mean()
    joined = joined[joined.index.dayofweek == 1]
    
    return joined




In [4]:
start_year = 2018
start_month = 1
end_year = 2020
end_month = 10
index = "NDAQ"

for word in words:
    word_trends[word] = getStats(word, start_year, start_month, end_year, end_month, index)

hedge:2018-01-01 2018-01-31
hedge:2018-02-01 2018-02-28
hedge:2018-03-01 2018-03-31
hedge:2018-04-01 2018-04-30
hedge:2018-05-01 2018-05-31
hedge:2018-06-01 2018-06-30
hedge:2018-07-01 2018-07-31
hedge:2018-08-01 2018-08-31
hedge:2018-09-01 2018-09-30
hedge:2018-10-01 2018-10-31
hedge:2018-11-01 2018-11-30
hedge:2018-12-01 2018-12-31
hedge:2019-01-01 2019-01-31
hedge:2019-02-01 2019-02-28
hedge:2019-03-01 2019-03-31
hedge:2019-04-01 2019-04-30
hedge:2019-05-01 2019-05-31
hedge:2019-06-01 2019-06-30
hedge:2019-07-01 2019-07-31
hedge:2019-08-01 2019-08-31
hedge:2019-09-01 2019-09-30
hedge:2019-10-01 2019-10-31
hedge:2019-11-01 2019-11-30
hedge:2019-12-01 2019-12-31
hedge:2020-01-01 2020-01-31
hedge:2020-02-01 2020-02-29
hedge:2020-03-01 2020-03-31
hedge:2020-04-01 2020-04-30
hedge:2020-05-01 2020-05-31
hedge:2020-06-01 2020-06-30
hedge:2020-07-01 2020-07-31
hedge:2020-08-01 2020-08-31
hedge:2020-09-01 2020-09-30
hedge:2020-10-01 2020-10-31
[*********************100%**********************

In [17]:
def ourFunc(df):
    return pd.Series(df['change'])

def ourFunc1(df):
    return pd.Series(df['change moving avg'])

In [18]:
from backtesting import Strategy
from backtesting.lib import crossover


class ourStrat(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    #n1 = 10
    #n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        #self.sma1 = self.I(SMA, self.data.Close, self.n1)
        #self.sma2 = self.I(SMA, self.data.Close, self.n2)
        self.change = self.I(ourFunc1, self.data)
        self.own = False
        #self.buy_from_last_week = False
        #self.sell_from_last_week = True
    
    def next(self):
        """
        if self.buy_from_last_week:
            self.buy()
            self.own = True
            self.buy_from_last_week = False
        
        elif self.sell_from_last_week:
            self.sell()
            self.own = False
            self.sell_from_last_week = False
        """
        
        if self.change[-1] > 0: #and not self.own: #crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()
            #self.own = True

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif self.change[-1] < 0: #and self.own: #crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()
            #self.own = False

In [19]:
returns = {}

for word in word_trends.keys():
    bt = Backtest(word_trends[word], ourStrat, cash=100_000_000, commission=0)
    stats = bt.run()
    returns[word] = stats[6]
    returns['BENCHMARK'] = stats[7]

returns

{'hedge': -63.394882864974974,
 'BENCHMARK': 58.170446484627625,
 'dividend': 50.42751747047425,
 'earnings': 17.173222357566832,
 'inflation': -1.4651252545471192,
 'markets': 12.762566652565003,
 'bonds': -30.6086030111084,
 'debt': 41.73815968918609,
 'financial markets': -25.83769240019989,
 'gains': -0.27170025929260255,
 'investment': 47.403307242813106}

In [25]:
import random

# randomly buy and sell 
class randomStrat(Strategy):
    def init(self):
        self.change = self.I(ourFunc1, self.data)
    def next(self):
        if random.choice([True, False]):
            self.position.close()
            self.buy()
        else:
            self.position.close()
            self.sell()

In [57]:
random_returns = []
for x in range(201):
    bt = Backtest(word_trends["debt"], randomStrat, cash=100_000_000, commission=0)
    random_stats = bt.run()
    # returns["RANDOM"] = random_stats[6]
    random_returns.append(random_stats[6])

In [58]:
np.mean(random_returns)

0.6015814803052306

In [28]:
stats

Start                     2018-01-02 00:00:00
End                       2020-09-29 00:00:00
Duration                   1001 days 00:00:00
Exposure Time [%]                     97.8873
Equity Final [$]                  1.47403e+08
Equity Peak [$]                   1.63725e+08
Return [%]                            47.4033
Buy & Hold Return [%]                 58.1704
Return (Ann.) [%]                     99.0861
Volatility (Ann.) [%]                 105.498
Sharpe Ratio                         0.939219
Sortino Ratio                         2.99569
Calmar Ratio                          3.29556
Max. Drawdown [%]                    -30.0666
Avg. Drawdown [%]                    -6.15944
Max. Drawdown Duration      294 days 00:00:00
Avg. Drawdown Duration       63 days 00:00:00
# Trades                                  139
Win Rate [%]                          61.1511
Best Trade [%]                         29.398
Worst Trade [%]                      -19.2485
Avg. Trade [%]                    